In [2]:
"""
FFIEC Neural Network Anomaly Detection (Step 5b Track 2 - Per-Bank)
====================================================================
Purpose: Detect anomalous quarters for each bank INDIVIDUALLY using
         a neural network autoencoder + LOF ensemble, trained on that
         bank's own historical QoQ changes.

Track 2 Philosophy:
  - Each bank gets its OWN model trained only on its own history
  - "Anomalous" = this quarter is unusual for THIS bank specifically
  - Contrast with Track 1 where anomalous = unusual vs. all 6 banks

Architecture adapted for small per-bank samples (~93 obs each):
  - Shallower bottleneck: Input -> 16 -> 8 -> 16 -> Output
  - Heavier L2 regularization (alpha=0.05)
  - More aggressive PCA (85% variance retention)
  - Larger validation fraction (20%)
  - Reduced LOF neighbors (12)

Input:  per_bank_qoq/ffiec_<bank_name>_qoq.csv  (from Step 4 Track 2)
Output: per_bank_nn/ffiec_<bank_name>_nn_anomalies.csv       (all obs w/ scores)
        per_bank_nn/ffiec_<bank_name>_nn_anomalies_flagged.csv (flagged only)
        per_bank_nn/nn_report_<bank_name>.txt                 (per-bank report)
        per_bank_nn/nn_cross_bank_summary.txt                 (comparison report)

Author: Wake Forest MSBA Practicum Team 4
Date: February 2026
"""

import pandas as pd
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import LocalOutlierFactor
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


# =============================================================================
# CONFIGURATION
# =============================================================================

INPUT_DIR = Path("per_bank_qoq")
OUTPUT_DIR = Path("per_bank_nn")

# Column identifiers
QUARTER_COLUMN = "quarter"

# Neural Network Architecture — adapted for ~93 obs per bank
# Smaller bottleneck to prevent memorization with tiny samples
HIDDEN_LAYERS = (16, 8, 16)
ALPHA = 0.05              # Heavier L2 reg than Track 1 (was 0.01)
MAX_ITER = 500
LEARNING_RATE = 0.001
RANDOM_STATE = 42

# PCA — balance between compression and information retention
PCA_VARIANCE = 0.95       # Retain 95% variance (was 0.80, which collapsed to 1 component)
PCA_MIN_COMPONENTS = 10   # Always keep at least 10 components regardless of variance

# LOF — fewer neighbors for small samples
LOF_NEIGHBORS = 12        # Was 20 in Track 1 (20/93 = 21% is too much)
LOF_CONTAMINATION = 0.05

# Anomaly Detection
CONTAMINATION = 0.05      # Flag top 5% as anomalies

# Ensemble weights
NN_WEIGHT = 0.6
LOF_WEIGHT = 0.4

# Bank name lookup (IDRSSD -> display name)
BANKS = {
    480228:  'Bank of America',
    852218:  'JPMorgan Chase Bank',
    451965:  'Wells Fargo Bank',
    476810:  'Citibank',
    1456501: 'Morgan Stanley Bank',
    2182786: 'Goldman Sachs Bank USA',
}


# =============================================================================
# AUTOENCODER CLASS
# =============================================================================

class ShallowAutoencoder:
    """
    Shallow autoencoder for per-bank anomaly detection.

    Adapted for small sample sizes (~93 obs):
      - 8-dim bottleneck (vs 32 in Track 1)
      - Heavier regularization
      - Larger validation fraction
      - More aggressive PCA preprocessing
    """

    def __init__(self, hidden_layers=(16, 8, 16), alpha=0.05,
                 max_iter=500, learning_rate=0.001, random_state=42,
                 use_pca=True, pca_variance=0.95, pca_min_components=10):

        self.hidden_layers = hidden_layers
        self.alpha = alpha
        self.use_pca = use_pca
        self.pca_variance = pca_variance
        self.pca_min_components = pca_min_components

        self.scaler = RobustScaler()
        self.pca = None  # Will be created in fit() after determining n_components

        self.model = MLPRegressor(
            hidden_layer_sizes=hidden_layers,
            activation='relu',
            solver='adam',
            alpha=alpha,
            learning_rate='adaptive',
            learning_rate_init=learning_rate,
            max_iter=max_iter,
            early_stopping=True,
            validation_fraction=0.20,   # Larger than Track 1's 0.15
            n_iter_no_change=30,
            random_state=random_state,
            verbose=False
        )
        self.is_fitted = False
        self.n_components_ = None

    def fit(self, X):
        """Train autoencoder with optional PCA preprocessing."""
        X_scaled = self.scaler.fit_transform(X)

        if self.use_pca:
            # First pass: determine how many components for the target variance
            pca_probe = PCA(n_components=self.pca_variance, random_state=42)
            pca_probe.fit(X_scaled)
            n_variance_components = pca_probe.n_components_

            # Apply minimum floor, but don't exceed sample size or feature count
            max_possible = min(X_scaled.shape[0], X_scaled.shape[1])
            n_components = max(n_variance_components, self.pca_min_components)
            n_components = min(n_components, max_possible - 1)  # Leave room for validation

            self.pca = PCA(n_components=n_components, random_state=42)
            X_transformed = self.pca.fit_transform(X_scaled)
            self.n_components_ = X_transformed.shape[1]
        else:
            X_transformed = X_scaled
            self.n_components_ = X_transformed.shape[1]

        # Train: reconstruct input from itself
        self.model.fit(X_transformed, X_transformed)
        self.is_fitted = True
        return self

    def transform(self, X):
        """Transform data through scaling and optional PCA."""
        X_scaled = self.scaler.transform(X)
        if self.use_pca:
            return self.pca.transform(X_scaled)
        return X_scaled

    def get_reconstruction_error(self, X):
        """Compute per-observation reconstruction error (MSE)."""
        X_transformed = self.transform(X)
        X_reconstructed = self.model.predict(X_transformed)
        mse = np.mean((X_transformed - X_reconstructed) ** 2, axis=1)
        return mse

    def score_anomalies(self, X, contamination=0.05):
        """Score and flag anomalies based on reconstruction error."""
        errors = self.get_reconstruction_error(X)

        # Normalize to 0-100 scale
        scores = 100 * (errors - errors.min()) / (errors.max() - errors.min() + 1e-10)

        # Flag top contamination%
        threshold = np.percentile(errors, 100 * (1 - contamination))
        is_anomaly = errors >= threshold

        return scores, is_anomaly, errors


# =============================================================================
# LOCAL OUTLIER FACTOR
# =============================================================================

def run_lof(X, n_neighbors=12, contamination=0.05):
    """
    Run Local Outlier Factor for comparison.
    Catches context-dependent outliers that global methods miss.
    """
    # Ensure n_neighbors doesn't exceed sample size - 1
    effective_neighbors = min(n_neighbors, X.shape[0] - 1)

    lof = LocalOutlierFactor(
        n_neighbors=effective_neighbors,
        contamination=contamination,
        novelty=False
    )

    labels = lof.fit_predict(X)

    # Negative outlier factor (more negative = more anomalous)
    scores_raw = -lof.negative_outlier_factor_

    # Normalize to 0-100
    scores = 100 * (scores_raw - scores_raw.min()) / (scores_raw.max() - scores_raw.min() + 1e-10)

    is_anomaly = labels == -1

    return scores, is_anomaly


# =============================================================================
# ENSEMBLE SCORING
# =============================================================================

def compute_ensemble_score(nn_scores, lof_scores, nn_weight=0.6, lof_weight=0.4):
    """Combine NN and LOF scores with NN weighted higher."""
    return nn_weight * nn_scores + lof_weight * lof_scores


# =============================================================================
# PER-BANK REPORT GENERATION
# =============================================================================

def generate_bank_report(bank_name, df, df_flagged, feature_cols,
                         autoencoder, training_info, lof_info):
    """Generate a detailed anomaly report for one bank."""
    lines = [
        "=" * 70,
        f"NEURAL NETWORK ANOMALY DETECTION REPORT: {bank_name.upper()}",
        f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
        "=" * 70,
        "",
        "DATA CHARACTERISTICS",
        "-" * 40,
        f"Observations (quarters): {len(df)}",
        f"Original QoQ features: {len(feature_cols)}",
        f"Obs/feature ratio: {len(df)/max(len(feature_cols),1):.2f}",
        f"PCA components ({int(PCA_VARIANCE*100)}% var): {autoencoder.n_components_}",
        f"Obs/PCA-component ratio: {len(df)/max(autoencoder.n_components_,1):.2f}",
        "",
        "MODEL ARCHITECTURE (Track 2 - Per-Bank, Small Sample)",
        "-" * 40,
        f"Type: Shallow Autoencoder",
        f"Hidden layers: {HIDDEN_LAYERS}",
        f"Bottleneck dimension: {HIDDEN_LAYERS[1]}",
        f"L2 regularization (alpha): {ALPHA}",
        f"PCA preprocessing: Yes ({autoencoder.n_components_} components)",
        f"Validation fraction: 20%",
        "",
        "TRAINING INFO",
        "-" * 40,
        f"Iterations: {training_info['n_iter']}",
        f"Final loss: {training_info['loss']:.6f}",
        f"Converged: {training_info['converged']}",
        "",
        "ENSEMBLE APPROACH",
        "-" * 40,
        f"  1. Shallow Autoencoder ({int(NN_WEIGHT*100)}% weight)",
        f"  2. Local Outlier Factor ({int(LOF_WEIGHT*100)}% weight)",
        f"  LOF neighbors: {LOF_NEIGHBORS}",
        f"  LOF anomalies: {lof_info['n_anomalies']}",
        "",
        "RESULTS",
        "-" * 40,
        f"Total quarters analyzed: {len(df)}",
        f"Anomalies flagged (ensemble): {len(df_flagged)} ({len(df_flagged)/max(len(df),1):.1%})",
        "",
    ]

    # List all flagged anomalies for this bank
    if len(df_flagged) > 0:
        lines.append("FLAGGED ANOMALOUS QUARTERS")
        lines.append("-" * 40)
        for _, row in df_flagged.iterrows():
            lines.append(
                f"  {row['ensemble_score']:.1f}: {row[QUARTER_COLUMN]}"
                f"  (NN: {row['nn_score']:.1f}, LOF: {row['lof_score']:.1f})"
            )
        lines.append("")

    # Check stress period coverage
    lines.append("VALIDATION: STRESS PERIOD COVERAGE")
    lines.append("-" * 40)
    lines.append("Known stress periods that should show elevated scores:")
    lines.append("  - 2008-2009: Financial Crisis")
    lines.append("  - 2020: COVID-19 pandemic")
    lines.append("")

    stress_periods = ['2008', '2009', '2010', '2020']
    for period in stress_periods:
        period_mask = df[QUARTER_COLUMN].astype(str).str.contains(period)
        period_data = df[period_mask]
        if len(period_data) > 0:
            n_flagged = period_data['is_anomaly'].sum()
            avg_score = period_data['ensemble_score'].mean()
            lines.append(f"  {period}: {n_flagged} flagged, avg score {avg_score:.1f}")
        else:
            lines.append(f"  {period}: no data")

    return "\n".join(lines)


# =============================================================================
# CROSS-BANK SUMMARY REPORT
# =============================================================================

def generate_cross_bank_summary(all_results):
    """Generate a comparison report across all banks."""
    lines = [
        "=" * 70,
        "NEURAL NETWORK ANOMALY DETECTION: CROSS-BANK SUMMARY (Track 2)",
        f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
        "=" * 70,
        "",
        "APPROACH",
        "-" * 40,
        "Each bank has its own autoencoder trained only on its own history.",
        "Anomalies reflect quarters unusual for THAT bank, not cross-bank.",
        f"Architecture: {HIDDEN_LAYERS} | Alpha: {ALPHA} | PCA: {int(PCA_VARIANCE*100)}%",
        "",
        "PER-BANK OVERVIEW",
        "-" * 70,
        f"{'Bank':<30} {'Obs':>5} {'Feat':>5} {'PCA':>5} {'Anom':>5} {'MaxScore':>9}",
        "-" * 70,
    ]

    for bank_name, res in all_results.items():
        df = res['df_all']
        n_anom = df['is_anomaly'].sum()
        max_score = df['ensemble_score'].max()
        lines.append(
            f"{bank_name:<30} {res['n_obs']:>5} {res['n_features']:>5} "
            f"{res['n_pca']:>5} {n_anom:>5} {max_score:>9.1f}"
        )

    lines.extend(["", ""])

    # Top anomalies across all banks
    lines.append("TOP 10 ANOMALIES ACROSS ALL BANKS")
    lines.append("-" * 70)

    all_flagged = []
    for bank_name, res in all_results.items():
        df = res['df_all'].copy()
        df['_bank_name'] = bank_name
        all_flagged.append(df)

    if all_flagged:
        combined = pd.concat(all_flagged, ignore_index=True)
        top10 = combined.nlargest(10, 'ensemble_score')
        for _, row in top10.iterrows():
            lines.append(
                f"  {row['ensemble_score']:.1f}: {row['_bank_name']} - {row[QUARTER_COLUMN]}"
                f"  (NN: {row['nn_score']:.1f}, LOF: {row['lof_score']:.1f})"
            )

    lines.extend(["", ""])

    # Stress period comparison
    lines.append("STRESS PERIOD COMPARISON")
    lines.append("-" * 70)
    lines.append(f"{'Bank':<30} {'2008':>6} {'2009':>6} {'2010':>6} {'2020':>6}")
    lines.append("-" * 70)

    for bank_name, res in all_results.items():
        df = res['df_all']
        counts = []
        for period in ['2008', '2009', '2010', '2020']:
            mask = df[QUARTER_COLUMN].astype(str).str.contains(period)
            n = df.loc[mask, 'is_anomaly'].sum() if mask.any() else 0
            counts.append(str(int(n)))
        lines.append(f"{bank_name:<30} {counts[0]:>6} {counts[1]:>6} {counts[2]:>6} {counts[3]:>6}")

    lines.extend(["", ""])

    # Model convergence comparison
    lines.append("MODEL CONVERGENCE")
    lines.append("-" * 70)
    lines.append(f"{'Bank':<30} {'Iters':>6} {'Loss':>10} {'Converged':>10}")
    lines.append("-" * 70)

    for bank_name, res in all_results.items():
        ti = res['training_info']
        lines.append(
            f"{bank_name:<30} {ti['n_iter']:>6} {ti['loss']:>10.6f} "
            f"{'Yes' if ti['converged'] else 'No':>10}"
        )

    return "\n".join(lines)


# =============================================================================
# PROCESS ONE BANK
# =============================================================================

def process_bank(filepath, bank_name):
    """
    Run the full autoencoder + LOF ensemble pipeline on a single bank's
    QoQ CSV and return all results.
    """
    print(f"\n{'='*70}")
    print(f"  {bank_name.upper()}")
    print(f"{'='*70}")

    # Load data (handle .xls files that are actually CSVs)
    try:
        df = pd.read_csv(filepath, low_memory=False)
    except Exception:
        df = pd.read_excel(filepath, engine='xlrd')
    print(f"  Loaded: {df.shape[0]} rows x {df.shape[1]} columns")

    # -------------------------------------------------------------------------
    # Detect data orientation and normalize to: rows=quarters, cols=features
    # New format: rows=features, cols=quarters, with 'feature' column
    # Old format: rows=quarters, cols=features, with 'IDRSSD'/'quarter'
    # Detection: if 'feature' column exists and contains _qoq values, transpose
    # -------------------------------------------------------------------------
    if 'feature' in df.columns:
        has_qoq_features = df['feature'].astype(str).str.contains('_qoq').any()
        if has_qoq_features:
            print("  Detected transposed format (features as rows). Transposing...")
            feature_names = df['feature'].tolist()
            df_t = df.drop(columns=['feature']).T
            df_t.columns = feature_names
            df_t.index.name = 'quarter'
            df_t = df_t.reset_index()
            df_t.rename(columns={'index': 'quarter'}, inplace=True)
            df = df_t
            print(f"  After transpose: {df.shape[0]} rows x {df.shape[1]} columns")

    # Get QoQ feature columns
    feature_cols = [c for c in df.columns if c.endswith('_qoq')]
    n_features = len(feature_cols)
    print(f"  QoQ features: {n_features}")
    print(f"  Quarters: {len(df)}")
    print(f"  Obs/feature ratio: {len(df)/max(n_features,1):.2f}")

    X = df[feature_cols].copy()

    # Force numeric (transposed data may have string types)
    X = X.apply(pd.to_numeric, errors='coerce')

    # Handle missing/infinite values
    X = X.replace([np.inf, -np.inf], np.nan)
    for col in feature_cols:
        if X[col].isnull().any():
            X[col] = X[col].fillna(X[col].median())

    # Check if we have enough data (quarters)
    n_quarters = len(df)
    if n_quarters < 10:
        print(f"  WARNING: Only {n_quarters} quarters. Results may be unreliable.")
    if n_features == 0:
        print(f"  ERROR: No QoQ feature columns found. Skipping.")
        return None

    # =========================================================================
    # MODEL 1: Shallow Autoencoder with PCA
    # =========================================================================
    print(f"\n  [1/2] Training Shallow Autoencoder...")
    print(f"    Architecture: {HIDDEN_LAYERS} ({HIDDEN_LAYERS[1]}-dim bottleneck)")
    print(f"    L2 regularization: {ALPHA}")
    print(f"    PCA: {int(PCA_VARIANCE*100)}% variance retention")

    autoencoder = ShallowAutoencoder(
        hidden_layers=HIDDEN_LAYERS,
        alpha=ALPHA,
        max_iter=MAX_ITER,
        learning_rate=LEARNING_RATE,
        random_state=RANDOM_STATE,
        use_pca=True,
        pca_variance=PCA_VARIANCE,
        pca_min_components=PCA_MIN_COMPONENTS
    )

    # Reduce validation fraction for very small samples (e.g., Goldman at 67 quarters)
    if n_quarters < 80:
        autoencoder.model.set_params(validation_fraction=0.15)
        print(f"    Validation fraction reduced to 15% (small sample)")

    autoencoder.fit(X)

    training_info = {
        'n_iter': autoencoder.model.n_iter_,
        'loss': autoencoder.model.loss_,
        'converged': autoencoder.model.n_iter_ < MAX_ITER
    }
    print(f"    PCA components: {autoencoder.n_components_}")
    print(f"    Training iterations: {training_info['n_iter']}")
    print(f"    Final loss: {training_info['loss']:.6f}")

    nn_scores, nn_anomaly, nn_errors = autoencoder.score_anomalies(X, CONTAMINATION)
    print(f"    NN anomalies: {nn_anomaly.sum()}")

    # =========================================================================
    # MODEL 2: Local Outlier Factor
    # =========================================================================
    print(f"\n  [2/2] Running Local Outlier Factor...")

    X_scaled = autoencoder.scaler.transform(X)

    # Apply PCA to LOF input as well for consistency
    if autoencoder.use_pca:
        X_lof = autoencoder.pca.transform(X_scaled)
    else:
        X_lof = X_scaled

    lof_scores, lof_anomaly = run_lof(X_lof, LOF_NEIGHBORS, LOF_CONTAMINATION)
    lof_info = {'n_anomalies': int(lof_anomaly.sum())}
    print(f"    LOF anomalies: {lof_info['n_anomalies']}")

    # =========================================================================
    # ENSEMBLE
    # =========================================================================
    print(f"\n  Combining scores (NN: {int(NN_WEIGHT*100)}%, LOF: {int(LOF_WEIGHT*100)}%)...")
    ensemble_scores = compute_ensemble_score(nn_scores, lof_scores, NN_WEIGHT, LOF_WEIGHT)

    threshold = np.percentile(ensemble_scores, 100 * (1 - CONTAMINATION))
    ensemble_anomaly = ensemble_scores >= threshold

    # Add scores to dataframe
    df['nn_score'] = nn_scores
    df['lof_score'] = lof_scores
    df['ensemble_score'] = ensemble_scores
    df['nn_anomaly'] = nn_anomaly
    df['lof_anomaly'] = lof_anomaly
    df['is_anomaly'] = ensemble_anomaly
    df['reconstruction_error'] = nn_errors
    df['anomaly_score'] = ensemble_scores  # backwards compatibility

    print(f"  Ensemble anomalies: {ensemble_anomaly.sum()}")

    both_flag = (nn_anomaly & lof_anomaly).sum()
    print(f"  Flagged by BOTH models: {both_flag} (high confidence)")

    df_flagged = df[df['is_anomaly']].sort_values('ensemble_score', ascending=False)

    # Generate per-bank report
    report = generate_bank_report(
        bank_name, df, df_flagged, feature_cols,
        autoencoder, training_info, lof_info
    )

    return {
        'df_all': df,
        'df_flagged': df_flagged,
        'feature_cols': feature_cols,
        'n_obs': len(df),
        'n_features': n_features,
        'n_pca': autoencoder.n_components_,
        'training_info': training_info,
        'lof_info': lof_info,
        'report': report,
    }


# =============================================================================
# MAIN
# =============================================================================

def main():
    OUTPUT_DIR.mkdir(exist_ok=True)

    print("\n" + "=" * 70)
    print("FFIEC NEURAL NETWORK ANOMALY DETECTION (Track 2 - Per-Bank)")
    print("Shallow Autoencoder + LOF Ensemble, Trained Individually")
    print("=" * 70)

    # Discover per-bank QoQ files (.csv or .xls that are actually CSVs)
    bank_files = sorted(
        list(INPUT_DIR.glob("ffiec_*_qoq.csv")) +
        list(INPUT_DIR.glob("ffiec_*_qoq.xls"))
    )

    if not bank_files:
        print(f"\nERROR: No per-bank QoQ files found in {INPUT_DIR}/")
        print("  Expected files like: ffiec_bank_of_america_qoq.csv (or .xls)")
        print("  Run Step 4 Track 2 first.")
        return None

    print(f"\nFound {len(bank_files)} per-bank QoQ files in {INPUT_DIR}/:")
    for f in bank_files:
        print(f"  {f.name}")

    # Process each bank
    all_results = {}

    for filepath in bank_files:
        # Derive bank slug and display name from filename
        # e.g., ffiec_bank_of_america_qoq.csv -> bank_of_america
        #   or  ffiec_bank_of_america_qoq.xls -> bank_of_america
        slug = filepath.stem.replace('ffiec_', '').replace('_qoq', '')
        bank_name = slug.replace('_', ' ').title()

        # Try to match to canonical name from BANKS dict
        for rssd, name in BANKS.items():
            candidate_slug = name.lower().replace(' ', '_').replace('.', '')
            if candidate_slug == slug:
                bank_name = name
                break

        result = process_bank(filepath, bank_name)

        if result is None:
            print(f"  SKIPPED: {bank_name} (insufficient data)")
            continue

        all_results[bank_name] = result

        # Save per-bank outputs
        out_all = OUTPUT_DIR / f"ffiec_{slug}_nn_anomalies.csv"
        out_flagged = OUTPUT_DIR / f"ffiec_{slug}_nn_anomalies_flagged.csv"
        out_report = OUTPUT_DIR / f"nn_report_{slug}.txt"

        result['df_all'].to_csv(out_all, index=False)
        result['df_flagged'].to_csv(out_flagged, index=False)
        with open(out_report, 'w') as f:
            f.write(result['report'])

        print(f"\n  Saved: {out_all.name} ({len(result['df_all'])} rows)")
        print(f"  Saved: {out_flagged.name} ({len(result['df_flagged'])} rows)")
        print(f"  Saved: {out_report.name}")

    # =========================================================================
    # CROSS-BANK SUMMARY
    # =========================================================================
    if all_results:
        print("\n\n" + "=" * 70)
        print("CROSS-BANK SUMMARY")
        print("=" * 70)

        summary = generate_cross_bank_summary(all_results)
        summary_path = OUTPUT_DIR / "nn_cross_bank_summary.txt"
        with open(summary_path, 'w') as f:
            f.write(summary)
        print(f"Saved: {summary_path.name}")

        # Print summary table
        print(f"\n{'Bank':<30} {'Obs':>5} {'Feat':>5} {'PCA':>5} {'Anom':>5} {'MaxScore':>9}")
        print("-" * 70)
        for bank_name, res in all_results.items():
            df = res['df_all']
            n_anom = df['is_anomaly'].sum()
            max_score = df['ensemble_score'].max()
            print(
                f"{bank_name:<30} {res['n_obs']:>5} {res['n_features']:>5} "
                f"{res['n_pca']:>5} {n_anom:>5} {max_score:>9.1f}"
            )

        # Top anomalies
        print(f"\nTOP 10 ANOMALIES ACROSS ALL BANKS:")
        all_dfs = []
        for bank_name, res in all_results.items():
            tmp = res['df_all'].copy()
            tmp['_bank_name'] = bank_name
            all_dfs.append(tmp)
        combined = pd.concat(all_dfs, ignore_index=True)
        for _, row in combined.nlargest(10, 'ensemble_score').iterrows():
            print(f"  {row['ensemble_score']:.1f}: {row['_bank_name']} - {row[QUARTER_COLUMN]}")

    print("\n" + "=" * 70)
    print("COMPLETE")
    print(f"All outputs saved to {OUTPUT_DIR.resolve()}/")
    print("=" * 70 + "\n")

    return all_results


if __name__ == "__main__":
    results = main()


FFIEC NEURAL NETWORK ANOMALY DETECTION (Track 2 - Per-Bank)
Shallow Autoencoder + LOF Ensemble, Trained Individually

Found 6 per-bank QoQ files in per_bank_qoq/:
  ffiec_bank_of_america_qoq.csv
  ffiec_citibank_qoq.csv
  ffiec_goldman_sachs_bank_usa_qoq.csv
  ffiec_jpmorgan_chase_bank_qoq.csv
  ffiec_morgan_stanley_bank_qoq.csv
  ffiec_wells_fargo_bank_qoq.csv

  BANK OF AMERICA
  Loaded: 430 rows x 99 columns
  Detected transposed format (features as rows). Transposing...
  After transpose: 98 rows x 431 columns
  QoQ features: 430
  Quarters: 98
  Obs/feature ratio: 0.23

  [1/2] Training Shallow Autoencoder...
    Architecture: (16, 8, 16) (8-dim bottleneck)
    L2 regularization: 0.05
    PCA: 95% variance retention
    PCA components: 10
    Training iterations: 454
    Final loss: 190824540.598304
    NN anomalies: 5

  [2/2] Running Local Outlier Factor...
    LOF anomalies: 5

  Combining scores (NN: 60%, LOF: 40%)...
  Ensemble anomalies: 5
  Flagged by BOTH models: 3 (high 